In [33]:
import numpy as np
import pandas as pd
import talib
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
import math
style.use("ggplot")
#plt.style.use('seaborn-white')
plt.rcParams["figure.figsize"] = (20,10)
#plt.rcParams['axes.facecolor'] = 'grey'
#myplot + theme(panel.background = element_rect(fill='green', colour='red'))
import yfinance as yf

In [34]:
current_time = dt.datetime.now()
print(current_time)

2021-03-15 00:13:49.265583


In [35]:
def BollingerBands (df0, period, dev):
    df = df0
    sma = df["Close"].rolling(window = period).mean()
    rstd = df["Close"].rolling(window = period).std()
    df["Standard Moving Average"] = sma
    df["std"] = rstd
    df["Upper Band"] = sma + dev * rstd
    df["Lower Band"] = sma - dev * rstd
    
    return df

In [36]:
def MACD(df0, short, long, signal):
    df = df0
    df["Short Term EMA"] = df["Close"].ewm(span=short, min_periods=short).mean()
    df["Long Term EMA"] = df["Close"].ewm(span=long, min_periods=long).mean()
    df["MACD Line"] = df["Short Term EMA"] - df["Long Term EMA"]
    df["Signal Line"] = df["MACD Line"].ewm(span=signal, min_periods=signal).mean()
    
    return df

In [37]:
def std(df0):
    df = df0
    close_sum = 0
    for i in range (len(df)):
        close_sum += df["Close"].iloc[i]
        
    close_mean = close_sum/len(df)
    variance_sum = 0
    for i in range (len(df)):
            variance_sum += ((close_mean - df["Close"].iloc[i]) * (close_mean - df["Close"].iloc[i]))
    
    variance = variance_sum/len(df)
    std = math.sqrt(variance)
    
    #print(close_sum)
    #print("\n")
    print(close_mean)
    #print("\n")
    print(variance_sum)
    #print("\n")
    #print(variance)
    print(std)

# Back Tester

# Strategy 2_1

In [38]:
stock = yf.Ticker("HINDALCO.NS").history(start = "2018-01-15", end = "2019-12-01", interval = "1d")  #28-01-2015 ---> 28-02-2015
stock

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2018-01-15,265.609173,269.983547,261.283908,262.168640,6592272,0.0,0
2018-01-16,263.249927,263.446541,255.287574,256.123108,8447591,0.0,0
2018-01-17,255.877351,260.743260,252.092788,258.285736,11037340,0.0,0
2018-01-18,259.219601,259.219601,247.816685,249.586090,11341249,0.0,0
2018-01-19,249.684402,254.943509,247.914982,253.616440,5898314,0.0,0
...,...,...,...,...,...,...,...
2019-11-25,190.523952,199.627310,189.379819,198.234451,16931103,0.0,0
2019-11-26,199.129862,201.865846,196.543123,197.687256,16163735,0.0,0
2019-11-27,198.383675,202.462784,198.383675,201.965332,7251212,0.0,0


In [39]:
dff = BollingerBands(stock, 13, 1.817) + MACD(stock, 7,42, 5)
dff['RSI_14'] = talib.RSI(dff['Close'], timeperiod=14)
df_test = dff.tail(400)
dff.tail(400)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Standard Moving Average,std,Upper Band,Lower Band,Short Term EMA,Long Term EMA,MACD Line,Signal Line,RSI_14
Date,,,,,,,,,,,,,,,,
2018-04-11,462.014441,464.963469,454.346979,460.441620,37604098,0.0,0,423.480480,18.069866,456.313428,390.647533,436.605166,443.582155,-6.976989,-17.160525,58.681538
2018-04-12,459.950128,463.194077,454.248685,457.296021,23425342,0.0,0,425.869951,20.371408,462.884799,388.855102,441.777880,444.260869,-2.482989,-12.264700,57.261966
2018-04-13,462.014459,468.895525,461.228063,467.519318,33549336,0.0,0,430.664011,22.339113,471.254179,390.073842,448.213239,445.408535,2.804704,-7.239298,60.598114
2018-04-16,466.339672,473.220738,463.095752,470.763214,18651030,0.0,0,435.034621,24.270237,479.133642,390.935600,453.850733,446.656104,7.194629,-2.426542,61.621821
2018-04-17,474.793569,485.213475,467.912503,470.664917,32835816,0.0,0,438.195371,26.106147,485.630240,390.760503,458.054279,447.834277,10.220002,1.789818,61.569620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-25,381.047904,399.254619,378.759639,396.468903,33862206,0.0,0,387.996920,12.292851,410.333031,365.660809,384.558427,381.119070,3.439357,2.769617,58.299392
2019-11-26,398.259725,403.731692,393.086247,395.374512,32327470,0.0,0,387.568348,11.894445,409.180555,365.956141,387.262448,381.782113,5.480335,3.673190,57.550056
2019-11-27,396.767350,404.925567,396.767350,403.930664,14502424,0.0,0,387.231610,11.305756,407.774168,366.689051,391.429502,382.812279,8.617223,5.321201,61.695348


In [40]:
#df_test = dff.tail(23)
tradebook = []
long_short = []
timestamp = []
count = 0
npl = 0
position_squared = True
target = 0
stoploss = 0
tradetime = "open"
# long/short info
# price at which bought/shorted
# date at which bought/shorted
# buy/sell info

# 1 for long, 0 for short

days_till_trade = 0

for i in range(len(df_test)-1):
    #print(openp)
    tradetime = "open"
    open_prev = df_test["Open"].iloc[i]
    openp = df_test["Open"].iloc[i+1]
    close = df_test["Close"].iloc[i]
    closep = df_test["Close"].iloc[i+1]
    #p = df_test["Pivot"].iloc[i+1]
    #r1 = df_test["R1"].iloc[i+1]
    #r2 = df_test["R2"].iloc[i+1]
    #r3 = df_test["R3"].iloc[i+1]
    #s1 = df_test["S1"].iloc[i+1]
    #s2 = df_test["S2"].iloc[i+1]
    #s3 = df_test["S3"].iloc[i+1]
    ub = df_test["Upper Band"].iloc[i]
    lb = df_test["Lower Band"].iloc[i]
    sma = df_test["Standard Moving Average"].iloc[i]
    std = df_test["std"].iloc[i]
    
    sema = df_test["Short Term EMA"].iloc[i-1]
    semap = df_test["Short Term EMA"].iloc[i]
    lema = df_test["Long Term EMA"].iloc[i-1]
    lemap = df_test["Long Term EMA"].iloc[i]
    
    if(position_squared == True and i < len(df_test)-2):
            #ub - openp < 0.117*std or       openp - lb < 0.117*std or
        if(((openp > ub) or (ub - openp < 0.097*std)) and tradetime == "open"):
            long_short.append(0)
            tradebook.append(openp)
            timestamp.append(df_test.index[i])
            npl += openp
            count += 1
            position_squared = False
            days_till_trade = 0

            print(npl)
            print("entered short\n")
        elif(((lb > openp) or (openp - lb < 0.097*std)) and tradetime == "open"):
            long_short.append(1)
            tradebook.append(openp)
            timestamp.append(df_test.index[i])
            npl -= openp
            count += 1
            position_squared = False
            days_till_trade = 0
            #tradetime = "close"

            print(npl)
            print("entered long\n")
    elif(position_squared == False):
        if((openp >= tradebook[count-1]) and long_short[count-1] == 1 and tradetime == "open"):
            long_short.append(0)
            tradebook.append(openp)
            timestamp.append(df_test.index[i])
            npl += openp
            count += 1
            position_squared = True
            print(npl)
            print("sold off and squared\n")

        elif((openp <= tradebook[count-1]) and long_short[count-1] == 0 and tradetime == "open"):
            long_short.append(1)
            tradebook.append(openp)
            timestamp.append(df_test.index[i])
            npl -= openp 
            count += 1
            position_squared = True
            print(npl)
            print("bought back and squared\n")

            
    
    tradetime = "close"
            
            
            
    if(position_squared == True and i < len(df_test)-2):
        if(((closep > ub) or (ub - closep < 0.097*std)) and tradetime == "close"):
            long_short.append(0)
            tradebook.append(closep)
            timestamp.append(df_test.index[i])
            npl += closep
            count += 1
            position_squared = False
            days_till_trade = 0
            #tradetime = "open"

            print(npl)
            print("entered short\n")
        elif(((lb > closep) or (closep - lb < 0.097*std)) and tradetime == "close"):
            long_short.append(1)
            tradebook.append(closep)
            timestamp.append(df_test.index[i])
            npl -= closep
            count += 1
            position_squared = False
            days_till_trade = 0
            #tradetime = "open"

            print(npl)
            print("entered long\n")
        
    elif(position_squared == False):
        if((closep >= tradebook[count-1]) and long_short[count-1] == 1 and tradetime == "close"):
            long_short.append(0)
            tradebook.append(closep)
            timestamp.append(df_test.index[i])
            npl += closep
            count += 1
            position_squared = True
            print(npl)
            print("sold off and squared\n")

        elif((closep <= tradebook[count-1]) and long_short[count-1] == 0 and tradetime == "close"):
            long_short.append(1)
            tradebook.append(closep)
            timestamp.append(df_test.index[i])
            npl -= closep 
            count += 1
            position_squared = True
            print(npl)
            print("bought back and squared\n")
        
        
        
    tradetime = "open"
    days_till_trade += 1
        
    if(i == len(df_test) - 2 and position_squared == False):           
        if(long_short[count-1] == 1):
            long_short.append(0)
            tradebook.append(closep)
            timestamp.append(df_test.index[i])
            npl += closep
            count += 1
            position_squared = True

            print(npl)
            print("sold off and squared\n")

        elif(long_short[count-1] == 0):
            long_short.append(1)
            tradebook.append(closep)
            timestamp.append(df_test.index[i])
            npl -= closep 
            count += 1
            position_squared = True 

            print(npl)
            print("bought back and squared\n")
    
    
    
        
print(npl)
print("\n")
print(tradebook)
print("\n")
print(timestamp)

459.95012809515583
entered short

2.654107587343333
bought back and squared

464.6685660918755
entered short

3.1455990336970103
bought back and squared

481.2814074718865
entered short

5.308201905480246
bought back and squared

484.8202258312615
entered short

6.586132394901995
bought back and squared

494.3554500218551
entered short

11.40291159501129
bought back and squared

-442.7474485124106
entered long

11.402925025479533
sold off and squared

461.71957541610453
entered short

15.728201404407798
bought back and squared

-403.1321196405141
entered long

28.409018054798423
sold off and squared

-389.1733794061391
entered long

31.55467052531219
sold off and squared

-356.9306810371878
entered long

36.961212483105214
sold off and squared

-347.69036832744166
entered long

37.84596775938519
sold off and squared

-347.7886452777242
entered long

39.51708057535308
sold off and squared

470.0752470792593
entered short

42.26949859418039
bought back and squared

494.94537139691477
ent

In [41]:
print(tradebook)
print("\n")
print(timestamp)
print("\n")
print(npl)

[459.95012809515583, 457.2960205078125, 462.0144585045322, 461.5229670581785, 478.1358084381895, 475.97320556640625, 479.51202392578125, 478.2340934363595, 487.7693176269531, 482.95253842684383, 454.1503601074219, 454.1503735378901, 450.316650390625, 445.99137401169673, 418.8603210449219, 431.5411376953125, 417.5823974609375, 420.72804993145127, 388.4853515625, 393.891893520293, 384.6515808105469, 385.53633608682685, 385.6346130371094, 387.30572585307726, 430.55816650390625, 427.80574848507894, 452.6758728027344, 439.81634521484375, 439.7175422932126, 443.1759948730469, 428.65047672657647, 430.8244076820028, 464.42077236190005, 453.05731201171875, 438.6305847167969, 441.0021057128906, 431.02203369140625, 432.99827912956545, 419.75731713571895, 430.6267395019531, 422.62293356173103, 432.0101623535156, 424.00628662109375, 426.67425537109375, 427.36592310400135, 426.7730407714844, 409.6784396203795, 411.7535095214844, 414.3226403314157, 407.30694580078125, 382.7024841308594, 388.334839118

# #Ignore the code below

# Strategy2_2

In [42]:
#df_test = dff.tail(23)
tradebook = []
long_short = []
timestamp = []
count = 0
npl = 0
position_squared = True
target = 0
stoploss = 0
tradetime = "open"
# long/short info
# price at which bought/shorted
# date at which bought/shorted
# buy/sell info

# 1 for long, 0 for short

days_till_trade = 0

for i in range(len(df_test)-1):
    #print(openp)
    tradetime = "open"
    open_prev = df_test["Open"].iloc[i]
    openp = df_test["Open"].iloc[i+1]
    close = df_test["Close"].iloc[i]
    closep = df_test["Close"].iloc[i+1]
    #p = df_test["Pivot"].iloc[i+1]
    #r1 = df_test["R1"].iloc[i+1]
    #r2 = df_test["R2"].iloc[i+1]
    #r3 = df_test["R3"].iloc[i+1]
    #s1 = df_test["S1"].iloc[i+1]
    #s2 = df_test["S2"].iloc[i+1]
    #s3 = df_test["S3"].iloc[i+1]
    ub = df_test["Upper Band"].iloc[i]
    lb = df_test["Lower Band"].iloc[i]
    sma = df_test["Standard Moving Average"].iloc[i]
    std = df_test["std"].iloc[i]
    
    sema = df_test["Short Term EMA"].iloc[i-1]
    semap = df_test["Short Term EMA"].iloc[i]
    lema = df_test["Long Term EMA"].iloc[i-1]
    lemap = df_test["Long Term EMA"].iloc[i]
    
    if(position_squared == True and i < len(df_test)-2):
            #ub - openp < 0.117*std or       openp - lb < 0.117*std or
        if(((openp > ub) or (ub - openp < 0.097*std)) and tradetime == "open"):
            long_short.append(0)
            tradebook.append(openp)
            timestamp.append(df_test.index[i])
            npl += openp
            count += 1
            position_squared = False
            days_till_trade = 0

            print(npl)
            print("entered short\n")
        elif(((lb > openp) or (openp - lb < 0.097*std)) and tradetime == "open"):
            long_short.append(1)
            tradebook.append(openp)
            timestamp.append(df_test.index[i])
            npl -= openp
            count += 1
            position_squared = False
            days_till_trade = 0
            #tradetime = "close"

            print(npl)
            print("entered long\n")
    elif(position_squared == False):
        if((openp >= tradebook[count-1] or (days_till_trade >= 7 and npl+openp > (npl+tradebook[count-1])*0.25)) and long_short[count-1] == 1 and tradetime == "open"):
            long_short.append(0)
            tradebook.append(openp)
            timestamp.append(df_test.index[i])
            npl += openp
            count += 1
            position_squared = True
            print(npl)
            print("sold off and squared\n")

        elif((openp <= tradebook[count-1] or (days_till_trade >= 7 and npl-openp > (npl-tradebook[count-1])*0.25)) and long_short[count-1] == 0 and tradetime == "open"):
            long_short.append(1)
            tradebook.append(openp)
            timestamp.append(df_test.index[i])
            npl -= openp 
            count += 1
            position_squared = True
            print(npl)
            print("bought back and squared\n")

            
    
    tradetime = "close"
            
            
            
    if(position_squared == True and i < len(df_test)-2):
        if(((closep > ub) or (ub - closep < 0.097*std)) and tradetime == "close"):
            long_short.append(0)
            tradebook.append(closep)
            timestamp.append(df_test.index[i])
            npl += closep
            count += 1
            position_squared = False
            days_till_trade = 0
            #tradetime = "open"

            print(npl)
            print("entered short\n")
        elif(((lb > closep) or (closep - lb < 0.097*std)) and tradetime == "close"):
            long_short.append(1)
            tradebook.append(closep)
            timestamp.append(df_test.index[i])
            npl -= closep
            count += 1
            position_squared = False
            days_till_trade = 0
            #tradetime = "open"

            print(npl)
            print("entered long\n")
        
    elif(position_squared == False):
        if((closep >= tradebook[count-1] or (days_till_trade >= 7 and npl+openp > (npl+tradebook[count-1])*0.25)) and long_short[count-1] == 1 and tradetime == "close"):
            long_short.append(0)
            tradebook.append(closep)
            timestamp.append(df_test.index[i])
            npl += closep
            count += 1
            position_squared = True
            print(npl)
            print("sold off and squared\n")

        elif((closep <= tradebook[count-1] or (days_till_trade >= 7 and npl-openp > (npl-tradebook[count-1])*0.25)) and long_short[count-1] == 0 and tradetime == "close"):
            long_short.append(1)
            tradebook.append(closep)
            timestamp.append(df_test.index[i])
            npl -= closep 
            count += 1
            position_squared = True
            print(npl)
            print("bought back and squared\n")
        
        
        
    tradetime = "open"
    days_till_trade += 1
        
    if(i == len(df_test) - 2 and position_squared == False):           
        if(long_short[count-1] == 1):
            long_short.append(0)
            tradebook.append(closep)
            timestamp.append(df_test.index[i])
            npl += closep
            count += 1
            position_squared = True

            print(npl)
            print("sold off and squared\n")

        elif(long_short[count-1] == 0):
            long_short.append(1)
            tradebook.append(closep)
            timestamp.append(df_test.index[i])
            npl -= closep 
            count += 1
            position_squared = True 

            print(npl)
            print("bought back and squared\n")
    
    
    
        
print(npl)
print("\n")
print(tradebook)
print("\n")
print(timestamp)

459.95012809515583
entered short

2.654107587343333
bought back and squared

464.6685660918755
entered short

3.1455990336970103
bought back and squared

481.2814074718865
entered short

5.308201905480246
bought back and squared

484.8202258312615
entered short

6.586132394901995
bought back and squared

494.3554500218551
entered short

11.40291159501129
bought back and squared

-442.7474485124106
entered long

11.402925025479533
sold off and squared

461.71957541610453
entered short

15.728201404407798
bought back and squared

-403.1321196405141
entered long

28.409018054798423
sold off and squared

-389.1733794061391
entered long

31.55467052531219
sold off and squared

-356.9306810371878
entered long

36.961212483105214
sold off and squared

-347.69036832744166
entered long

37.84596775938519
sold off and squared

-347.7886452777242
entered long

39.51708057535308
sold off and squared

470.0752470792593
entered short

42.26949859418039
bought back and squared

494.94537139691477
ent

In [43]:
print(tradebook)
print("\n")
print(timestamp)
print("\n")
print(npl)

[459.95012809515583, 457.2960205078125, 462.0144585045322, 461.5229670581785, 478.1358084381895, 475.97320556640625, 479.51202392578125, 478.2340934363595, 487.7693176269531, 482.95253842684383, 454.1503601074219, 454.1503735378901, 450.316650390625, 445.99137401169673, 418.8603210449219, 431.5411376953125, 417.5823974609375, 420.72804993145127, 388.4853515625, 393.891893520293, 384.6515808105469, 385.53633608682685, 385.6346130371094, 387.30572585307726, 430.55816650390625, 427.80574848507894, 452.6758728027344, 458.27903437895196, 485.0726623535156, 482.20710503825154, 486.15963776698965, 482.00946044921875, 453.8477783203125, 480.033203125, 501.47564697265625, 498.01719090213885, 439.81634521484375, 443.1759948730469, 428.65047672657647, 430.8244076820028, 464.42077236190005, 453.05731201171875, 438.6305847167969, 441.0021057128906, 431.02203369140625, 432.99827912956545, 419.75731713571895, 430.6267395019531, 422.62293356173103, 432.0101623535156, 424.00628662109375, 403.1567755470

In [44]:
print(std(df_test))

TypeError: 'numpy.float64' object is not callable

In [45]:
closep > tradebook[count-1]*1.0007 and 
closep < tradebook[count-1]*0.9993 and 

 and semap - sema > lemap - lema and semap > lemap
and semap - sema < lemap - lema and semap < lemap

SyntaxError: invalid syntax (<ipython-input-45-ee1377b10203>, line 1)